In [41]:
import numpy as np
import os
import csv
from random import random, sample, seed

from keras.preprocessing import sequence
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.models import Input, Model
from keras.layers import Dense, Embedding, GlobalAveragePooling1D, concatenate, Activation
from keras.layers.core import Masking, Dropout, Reshape
from keras.layers.normalization import BatchNormalization
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.callbacks import CSVLogger

In [5]:
data_path = 'data/submissions.csv'
embeddings_path = 'data/glove.6B.50d.txt'

In [11]:
titles = []
hours = []
minutes = []
dayofweeks = []
dayofyears = []
is_top_submission = []

max_rows = 16000 # my crappy computer can't handle too many rows

with open(data_path, 'r', encoding="latin1") as f:
    reader = csv.DictReader(f)
    i = 0
    for submission in reader:
        if i >= max_rows:
            break
        i += 1
        titles.append(submission['title'])
        hours.append(submission['hour'])
        minutes.append(submission['minute'])
        dayofweeks.append(submission['dayofweek'])
        dayofyears.append(submission['dayofyear'])
        is_top_submission.append(submission['is_top_submission'])
            
titles = np.array(titles)
hours = np.array(hours, dtype=int)
minutes = np.array(minutes, dtype=int)
dayofweeks = np.array(dayofweeks, dtype=int)
dayofyears = np.array(dayofyears, dtype=int)
is_top_submission = np.array(is_top_submission, dtype=int)

In [13]:
print(titles[0:2])
print(titles.shape)
print(hours[0:2])
print(minutes[0:2])
print(dayofweeks[0:2])
print(dayofyears[0:2])
print(is_top_submission[0:2])

['People who downloaded their Google data and went through it, what were the most unsettling things you found out they had stored about you?'
 "Have you ever felt you don't know/have forgotten who you really are? That you've spent years just adapting to surroundings to make life easier and don't know what's the real you anymore? If so, how did you overcome this?"]
(16000,)
[22  3]
[45 53]
[3 4]
[219 206]
[1 1]


My computer can't handle the actual top submissions.  Just make the second half of the "top" top submissions the real "top" submissions.

In [15]:
for i in range(max_rows//2, max_rows):
    is_top_submission[i] = 0

In [16]:
1 - np.mean(is_top_submission)

0.5

In [18]:
max_features = 40000

word_tokenizer = Tokenizer(max_features)
word_tokenizer.fit_on_texts(titles)

print(str(word_tokenizer.word_counts)[0:100])
print(str(word_tokenizer.word_index)[0:100])
print(len(word_tokenizer.word_counts))   # true word count

OrderedDict([('people', 1907), ('who', 1483), ('downloaded', 3), ('their', 430), ('google', 35), ('d
{'you': 1, 'what': 2, 'the': 3, 'to': 4, 'a': 5, 'of': 6, 'your': 7, 'is': 8, 'do': 9, 'in': 10, 'th
11462


In [19]:
titles_tf = word_tokenizer.texts_to_sequences(titles)

print(titles_tf[0])

[17, 23, 3681, 73, 641, 3126, 12, 566, 286, 16, 2, 49, 3, 27, 2181, 96, 1, 207, 58, 62, 57, 4540, 33, 1]


In [21]:
maxlen = 20
titles_tf = sequence.pad_sequences(titles_tf, maxlen=maxlen)

print(titles_tf[0])

[ 641 3126   12  566  286   16    2   49    3   27 2181   96    1  207
   58   62   57 4540   33    1]


In [22]:
embedding_vectors = {}

with open(embeddings_path, 'r') as f:
    for line in f:
        line_split = line.strip().split(" ")
        vec = np.array(line_split[1:], dtype=float)
        word = line_split[0]
        embedding_vectors[word] = vec
        
print(embedding_vectors['you'])

[-1.0919e-03  3.3324e-01  3.5743e-01 -5.4041e-01  8.2032e-01 -4.9391e-01
 -3.2588e-01  1.9972e-03 -2.3829e-01  3.5554e-01 -6.0655e-01  9.8932e-01
 -2.1786e-01  1.1236e-01  1.1494e+00  7.3284e-01  5.1182e-01  2.9287e-01
  2.8388e-01 -1.3590e+00 -3.7951e-01  5.0943e-01  7.0710e-01  6.2941e-01
  1.0534e+00 -2.1756e+00 -1.3204e+00  4.0001e-01  1.5741e+00 -1.6600e+00
  3.7721e+00  8.6949e-01 -8.0439e-01  1.8390e-01 -3.4332e-01  1.0714e-02
  2.3969e-01  6.6748e-02  7.0117e-01 -7.3702e-01  2.0877e-01  1.1564e-01
 -1.5190e-01  8.5908e-01  2.2620e-01  1.6519e-01  3.6309e-01 -4.5697e-01
 -4.8969e-02  1.1316e+00]


In [23]:
weights_matrix = np.zeros((max_features + 1, 50))

for word, i in word_tokenizer.word_index.items():

    embedding_vector = embedding_vectors.get(word)
    if embedding_vector is not None and i <= max_features:
        weights_matrix[i] = embedding_vector

# index 0 vector should be all zeroes, index 1 vector should be the same one as above
print(weights_matrix[0:2,:])

[[ 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00]
 [-1.0919e-03  3.3324e-01  3.5743e-01 -5.4041e-01  8.2032e-01 -4.9391e-01
  -3.2588e-01  1.9972e-03 -2.3829e-01  3.5554e-01 -6.0655e-01  9.8932e-01
  -2.1786e-01  1.1236e-01  1.1494e+00  7.3284e-01  5.1182e-01  2.9287e-01
   2.8388e-01 -1.3590e+00 -3.7951e-01  5.0943e-01  7.0710e-01  6.2941e-01
   1.0534e+00 -2.1756e+00 -1.3204e+00  4.0001e-01  1.5741e+00 -1.6600e+00
   3.7721e+

In [24]:
dayofyears_tf = dayofyears - 1

print(dayofyears_tf[0:10])

[218 205 220 209 234 190 233 224 206 151]


In [26]:
batch_size = 32
embedding_dims = 50
epochs = 20

In [27]:
titles_input = Input(shape=(maxlen,), name='titles_input')
titles_embedding = Embedding(max_features + 1, embedding_dims, weights=[weights_matrix])(titles_input)
titles_pooling = GlobalAveragePooling1D()(titles_embedding)

Instructions for updating:
Colocations handled automatically by placer.


In [28]:
aux_output = Dense(1, activation='sigmoid', name='aux_out')(titles_pooling)

In [29]:
meta_embedding_dims = 64

hours_input = Input(shape=(1,), name='hours_input')
hours_embedding = Embedding(24, meta_embedding_dims)(hours_input)
hours_reshape = Reshape((meta_embedding_dims,))(hours_embedding)

dayofweeks_input = Input(shape=(1,), name='dayofweeks_input')
dayofweeks_embedding = Embedding(7, meta_embedding_dims)(dayofweeks_input)
dayofweeks_reshape = Reshape((meta_embedding_dims,))(dayofweeks_embedding)

minutes_input = Input(shape=(1,), name='minutes_input')
minutes_embedding = Embedding(60, meta_embedding_dims)(minutes_input)
minutes_reshape = Reshape((meta_embedding_dims,))(minutes_embedding)

dayofyears_input = Input(shape=(1,), name='dayofyears_input')
dayofyears_embedding = Embedding(366, meta_embedding_dims)(dayofyears_input)
dayofyears_reshape = Reshape((meta_embedding_dims,))(dayofyears_embedding)

In [30]:
merged = concatenate([titles_pooling, hours_reshape, dayofweeks_reshape, minutes_reshape, dayofyears_reshape])

hidden_1 = Dense(256, activation='relu')(merged)
hidden_1 = BatchNormalization()(hidden_1)

main_output = Dense(1, activation='sigmoid', name='main_out')(hidden_1)

In [31]:
model = Model(inputs=[titles_input,
                      hours_input,
                      dayofweeks_input,
                      minutes_input,
                      dayofyears_input], outputs=[main_output, aux_output])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
              loss_weights=[1, 0.2])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
titles_input (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
hours_input (InputLayer)        (None, 1)            0                                            
__________________________________________________________________________________________________
dayofweeks_input (InputLayer)   (None, 1)            0                                            
__________________________________________________________________________________________________
minutes_input (InputLayer)      (None, 1)            0                                            
____________________________________________________________________________________________

In [38]:
seed(123)
split = 0.2

# returns randomized indices with no repeats
idx = sample(range(titles_tf.shape[0]), titles_tf.shape[0])

titles_tf = titles_tf[idx, :]
hours = hours[idx]
dayofweeks = dayofweeks[idx]
minutes = minutes[idx]
dayofyears_tf = dayofyears_tf[idx]
is_top_submission = is_top_submission[idx]

In [40]:
print(1 - np.mean(is_top_submission[:(int(titles_tf.shape[0] * split))]))

0.49531250000000004


In [42]:
csv_logger = CSVLogger('training.csv')

In [43]:
model.fit([titles_tf, hours, dayofweeks, minutes, dayofyears_tf], [is_top_submission, is_top_submission],
          batch_size=batch_size,
          epochs=epochs,
          validation_split=split, callbacks=[csv_logger])

Instructions for updating:
Use tf.cast instead.
Train on 12800 samples, validate on 3200 samples
Epoch 1/20
12800/12800 [==============================] - 12s 935us/step - loss: 0.8582 - main_out_loss: 0.7185 - aux_out_loss: 0.6984 - main_out_accuracy: 0.5161 - aux_out_accuracy: 0.4971 - val_loss: 0.8367 - val_main_out_loss: 0.6975 - val_aux_out_loss: 0.6958 - val_main_out_accuracy: 0.5094 - val_aux_out_accuracy: 0.4884
Epoch 2/20
12800/12800 [==============================] - 10s 816us/step - loss: 0.8245 - main_out_loss: 0.6859 - aux_out_loss: 0.6930 - main_out_accuracy: 0.5596 - aux_out_accuracy: 0.5156 - val_loss: 0.8667 - val_main_out_loss: 0.7278 - val_aux_out_loss: 0.6945 - val_main_out_accuracy: 0.4950 - val_aux_out_accuracy: 0.5022
Epoch 3/20
12800/12800 [==============================] - 11s 855us/step - loss: 0.7962 - main_out_loss: 0.6583 - aux_out_loss: 0.6894 - main_out_accuracy: 0.6059 - aux_out_accuracy: 0.5379 - val_loss: 0.8827 - val_main_out_loss: 0.7438 - val_aux_ou

In [44]:
def encode_text(text, maxlen):
    encoded = word_tokenizer.texts_to_sequences([text])
    return sequence.pad_sequences(encoded, maxlen=maxlen)

In [50]:
input_text = "Which movie's plot would drastically change if you removed a letter from its title?"
encoded_text = encode_text(input_text, maxlen)
input_hour = np.array([15])
input_minute = np.array([46])
input_dayofweek = np.array([1])
input_dayofyear = np.array([16 - 1])

model.predict([encoded_text, input_hour, input_dayofweek, input_minute, input_dayofyear])

[array([[0.9999597]], dtype=float32), array([[0.7188623]], dtype=float32)]